In [1]:
import duckdb

In [6]:
cn = duckdb.connect()
cn.execute("CREATE or replace view v_order_header as select * from read_parquet('data/order_headers.parquet')")
cn.execute("CREATE or replace view v_order_detail as select * from read_parquet('data/order_details.parquet')")
cn.execute("create or replace view v_geo as select * from read_parquet('data/store_geography.parquet')")

In [9]:
cn.sql("from v_order_header limit 5").show()
cn.sql("from v_order_detail limit 5").show()
cn.sql("from v_geo limit 5").show()

┌──────────┬────────────┬──────────┬─────────────┬────────────┬────────────────┬───────────────┬────────┬────────┬────────────┐
│ order_id │ order_date │ store_id │ customer_id │   status   │ payment_method │   subtotal    │  tax   │ total  │ line_count │
│  int64   │    date    │  int32   │    int32    │  varchar   │    varchar     │ decimal(38,2) │ double │ double │   int64    │
├──────────┼────────────┼──────────┼─────────────┼────────────┼────────────────┼───────────────┼────────┼────────┼────────────┤
│        1 │ 2025-05-06 │       18 │        4090 │ DELIVERED  │ CREDIT_CARD    │        435.80 │  34.86 │ 470.66 │          7 │
│        2 │ 2025-11-07 │       30 │        5782 │ NEW        │ CREDIT_CARD    │        237.90 │  19.03 │ 256.93 │          3 │
│        3 │ 2025-02-08 │       43 │        9411 │ DELIVERED  │ CREDIT_CARD    │        502.32 │  40.19 │ 542.51 │          7 │
│        4 │ 2025-06-13 │       27 │        2219 │ DELIVERED  │ DEBIT_CARD     │        181.91 │  14.55 

In [23]:
cn.sql("select order_id from v_order_header group by all having count(*)>=2")
cn.sql("select count(*) from v_order_header").show()
cn.sql("select count(*) from v_order_detail").show()
cn.sql("select count(*) from v_geo").show()

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│        10000 │
└──────────────┘

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│        44804 │
└──────────────┘

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│           50 │
└──────────────┘



In [16]:
sql = """
    select oh.order_id, od.product_id, g.state
    from v_order_header oh
    join v_order_detail od on oh.order_id = od.order_id
    join v_geo g on oh.store_id = g.store_id
    limit 10
"""
cn.sql(sql).show()

┌──────────┬────────────┬─────────┐
│ order_id │ product_id │  state  │
│  int64   │   int32    │ varchar │
├──────────┼────────────┼─────────┤
│        1 │          1 │ TX      │
│        2 │          5 │ TX      │
│        3 │          6 │ TX      │
│        3 │         13 │ TX      │
│        3 │         17 │ TX      │
│        4 │         26 │ TX      │
│        5 │          1 │ TX      │
│        5 │         12 │ TX      │
│        5 │         17 │ TX      │
│        6 │         18 │ TX      │
├──────────┴────────────┴─────────┤
│ 10 rows               3 columns │
└─────────────────────────────────┘



In [22]:
sql = """
    select g.state, count(distinct oh.order_id) as num_orders
    from v_order_header oh
        join v_geo g on oh.store_id = g.store_id
    group by g.state
    limit 10
"""
cn.sql(sql).show()

┌─────────┬────────────┐
│  state  │ num_orders │
│ varchar │   int64    │
├─────────┼────────────┤
│ PA      │        620 │
│ NY      │        501 │
│ UT      │        414 │
│ MN      │        397 │
│ NC      │        216 │
│ WI      │        821 │
│ GA      │        620 │
│ AR      │        614 │
│ CT      │        419 │
│ WA      │        425 │
├─────────┴────────────┤
│ 10 rows    2 columns │
└──────────────────────┘

